In [5]:
import tensorflow as tf
import transformers
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import os
sys.path.append('..')
import utils
import numpy as np
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [3]:
# Version Info
print("Tensforflow Version : " ,tf.__version__)
print("Transformers Version : " ,transformers.__version__)

Tensforflow Version :  2.12.0
Transformers Version :  4.35.0


In [35]:
# make a data folder
!mkdir -p data

# download the data
out_path = tf.keras.utils.get_file(origin="https://archive.ics.uci.edu/static/public/331/sentiment+labelled+sentences.zip",extract=True,cache_dir="data")
print("\n",out_path)


 data\datasets\sentiment+labelled+sentences.zip


A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file data already exists.
Error occurred while processing: data.


In [6]:
# Define the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Define the data directory path
data_dir = os.path.join(parent_dir, 'data', 'text_entailment_dataset')

# Define file paths
train_csv_path = os.path.join(data_dir, 'train.csv')
train_data_csv_path = os.path.join(data_dir, 'train_data.csv')
validation_data_csv_path = os.path.join(data_dir, 'validation_data.csv')
test_data_csv_path = os.path.join(data_dir, 'test_data.csv')

# Read the CSV file into a DataFrame
df = pd.read_csv(train_csv_path)

# Shuffle the DataFrame
df = df.sample(frac=1, random_state=42)

# Split the data into training and validation sets
train_df, validation_df = train_test_split(df, test_size=0.2, random_state=42)

# Further split training set into train and test subsets
train_df, test_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Write the training and validation DataFrames to separate CSV files
train_df.to_csv(train_data_csv_path, index=False)
validation_df.to_csv(validation_data_csv_path, index=False)
test_df.to_csv(test_data_csv_path, index=False)

# Read datasets from CSV files
train_dataset = pd.read_csv(train_data_csv_path)
validation_dataset = pd.read_csv(validation_data_csv_path)
test_dataset = pd.read_csv(test_data_csv_path)

In [7]:
train_dataset.shape

(351590, 3)

In [8]:
validation_dataset.head()

,premise,hypothesis,label
0,A young couple looking at different types and ...,a couple is looking,0
1,A man in a purple had climbs a rocky wall with...,A man is going to the temple.,2
2,A group of friends playing cards and trying to...,The people are playing a card game.,0
3,People looking at fish at an aquarium.,People looking at fish.,0
4,An older lady blowing out a 9 and a 0 number c...,A woman drinks coffee while driving.,2


In [39]:
validation_dataset.shape


(54937, 3)

In [40]:
test_dataset.head()


,premise,hypothesis
0,This church choir sings to the masses as they ...,The church has cracks in the ceiling.
1,This church choir sings to the masses as they ...,The church is filled with song.
2,This church choir sings to the masses as they ...,A choir singing at a baseball game.
3,"A woman with a green headscarf, blue shirt and...",The woman is young.
4,"A woman with a green headscarf, blue shirt and...",The woman is very happy.


In [41]:
test_dataset.shape

(9824, 2)

In [42]:
# define a max length constant
MAX_LENGTH = 64

In [43]:
from transformers import DistilBertTokenizer, DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
bert = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [44]:
print(tokenizer)
print(bert)

DistilBertTokenizer(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position

In [45]:
train_dataset[["premise"]] = train_dataset[["premise"]].astype(str)
train_dataset["premise"] = train_dataset["premise"].apply(utils.change_lower)
train_dataset["premise"] = train_dataset["premise"].apply(utils.clean_data)
train_dataset["premise"] = train_dataset["premise"].apply(utils.remover)

train_dataset[["hypothesis"]] = train_dataset[["hypothesis"]].astype(str)
train_dataset["hypothesis"] = train_dataset["hypothesis"].apply(utils.change_lower)
train_dataset["hypothesis"] = train_dataset["hypothesis"].apply(utils.clean_data)
train_dataset["hypothesis"] = train_dataset["hypothesis"].apply(utils.remover)

validation_dataset[["premise"]] = validation_dataset[["premise"]].astype(str)
validation_dataset["premise"] = validation_dataset["premise"].apply(utils.change_lower)
validation_dataset["premise"] = validation_dataset["premise"].apply(utils.clean_data)
validation_dataset["premise"] = validation_dataset["premise"].apply(utils.remover)

validation_dataset[["hypothesis"]] = validation_dataset[["hypothesis"]].astype(str)
validation_dataset["hypothesis"] = validation_dataset["hypothesis"].apply(utils.change_lower)
validation_dataset["hypothesis"] = validation_dataset["hypothesis"].apply(utils.clean_data)
validation_dataset["hypothesis"] = validation_dataset["hypothesis"].apply(utils.remover)

test_dataset[["premise"]] = test_dataset[["premise"]].astype(str)
test_dataset["premise"] = test_dataset["premise"].apply(utils.change_lower)
test_dataset["premise"] = test_dataset["premise"].apply(utils.clean_data)
test_dataset["premise"] = test_dataset["premise"].apply(utils.remover)

test_dataset[["hypothesis"]] = test_dataset[["hypothesis"]].astype(str)
test_dataset["hypothesis"] = test_dataset["hypothesis"].apply(utils.change_lower)
test_dataset["hypothesis"] = test_dataset["hypothesis"].apply(utils.clean_data)
test_dataset["hypothesis"] = test_dataset["hypothesis"].apply(utils.remover)

In [46]:
X_train = train_dataset['premise'] + train_dataset['hypothesis']
X_Val = validation_dataset['premise'] + validation_dataset['hypothesis']
X_test = test_dataset['premise'] + test_dataset['hypothesis']

In [47]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [48]:
# define a batch size for our experiments
BATCH_SIZE = 64
# define a percentage of the data to use for training

train_dataset = df = pd.read_csv('data/text_entailment_dataset/train_data.csv')
validation_dataset = df = pd.read_csv('data/text_entailment_dataset/validation_data.csv')
test_dataset = df = pd.read_csv('data/text_entailment_dataset/test_data.csv')



In [49]:
train_sentences = [train_dataset.loc[s][0] + train_dataset.loc[s][1] for s in range(len(train_dataset))]
train_labels = [train_dataset.loc[l][2] for l in range(len(train_dataset))]

validation_sentences = [validation_dataset.loc[s][0] + validation_dataset.loc[s][1] for s in range(len(validation_dataset))]
validation_labels = [validation_dataset.loc[l][2] for l in range(len(validation_dataset))]

test_sentences = [test_dataset.loc[s][0] + test_dataset.loc[s][1] for s in range(len(test_dataset))]

print("LENGTHS // Train sentences: " + str(len(train_sentences)) + ". Train labels: " + str(len(train_labels)))
print("LENGTHS // Test sentences: " + str(len(validation_sentences)) + ". Test labels: " + str(len(validation_labels)))
print("LENGTHS // Test sentences: " + str(len(test_sentences)))

C:\Users\chris\AppData\Local\Temp\ipykernel_15616\67804960.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  train_sentences = [train_dataset.loc[s][0] + train_dataset.loc[s][1] for s in range(len(train_dataset))]
C:\Users\chris\AppData\Local\Temp\ipykernel_15616\67804960.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  train_labels = [train_dataset.loc[l][2] for l in range(len(train_dataset))]
C:\Users\chris\AppData\Local\Temp\ipykernel_15616\67804960.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent w

LENGTHS // Train sentences: 494424. Train labels: 494424
LENGTHS // Test sentences: 54937. Test labels: 54937
LENGTHS // Test sentences: 9824


In [50]:
train_labels = train_dataset["label"]
validation_labels = validation_dataset["label"]

# Convert to one-hot encoded format
num_classes = len(set(train_labels))  # Calculate the number of classes

train_labels = to_categorical(train_labels, num_classes=num_classes)
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

print("train label shape:", train_labels.shape)
print("val label shape:", validation_labels.shape)

steps_per_epoch = len(train_labels) // BATCH_SIZE
validation_steps = len(validation_labels) // BATCH_SIZE


train label shape: (494424, 3)
val label shape: (54937, 3)


In [51]:
#Implementing KMeans clustering for better labels
from sklearn.feature_extraction.text import TfidfVectorizer

# Use a tfidf vectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train_sentences)

# Choosing 3 clusters
k = 3

# Apply K-means clustering
kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(X)

# Get cluster assignments for the training data
train_cluster_labels = kmeans.labels_
train_labels = to_categorical(train_cluster_labels, num_classes=num_classes)

In [52]:
# data generator for the model
def data_generator(sentences: np.array,labels: np.array,batch_size: int) -> (dict,tf.Tensor):
    i = 0
    start_idx = -1 * batch_size
    end_idx = 0

    while True:
        start_idx += batch_size
        end_idx += batch_size
        # TODO: append batch_size number of sentences and labels to batch_x and batch_y
        # Make sure that you don't re-use sentences and labels that you've already put into batches!

        if end_idx > len(sentences):
            end_idx = batch_size
            start_idx = 0

        batch_y = labels[start_idx:end_idx]

        # TODO: tokenize the batch_x, padding to MAX_LENGTH, and truncating to MAX_LENGTH
        batch_x = tokenizer(sentences[start_idx:end_idx], return_tensors="tf", max_length=MAX_LENGTH, truncation="longest_first", padding="max_length")

        # debugging prints (make sure that these are commented out when you actually train your model)
        # should be (batch_size, MAX_LENGTH)
        # print(batch_x['input_ids'].shape)

        # convert our ys into the appropriate tensor
        batch_y = tf.convert_to_tensor(batch_y)

        # debugging prints (make sure that these are commented out when you actually train your model)
        # should be (batch_size,)
        # print(batch_y.shape)
        yield dict(batch_x), batch_y

train_data = data_generator(train_sentences,train_labels,BATCH_SIZE)
val_data = data_generator(validation_sentences,validation_labels,BATCH_SIZE)

In [53]:
# TODO: Take a look at the contents of tmp_batch_x and tmp_batch_y and report the shapes of the `input_ids`
# and the y label tensor.
# make sure that the shapes are what you expect them to be
# (take a look at the comments in the data_generator code)

tmp_batch_x,tmp_batch_y = next(train_data)
val_batch_x, val_batch_y = next(val_data)

print(tmp_batch_x["input_ids"].shape)
print(tmp_batch_y.shape)
#print(tmp_batch_y)

print(val_batch_x["input_ids"].shape)
print(val_batch_y.shape)
#print(val_batch_y)
print(train_cluster_labels)


(64, 64)
(64, 3)
(64, 64)
(64, 3)
[2 2 2 ... 2 1 1]


In [54]:
from keras.src.callbacks import History
from transformers import TFDistilBertModel
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Define custom metrics functions
def precision(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    f1 = 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))
    return f1


In [55]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [56]:
#Get the number of GPUs needed
print("Total GPUs: ", strategy.num_replicas_in_sync)

Total GPUs:  1


In [57]:
#Prints the GPUs on your machine (if it's nvidia)
!nvidia-smi

Tue Apr 16 21:03:03 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.78                 Driver Version: 551.78         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   48C    P8             13W /  115W |      92MiB /   6144MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [58]:
#This will be our steps per epoch
print(len(train_sentences)//BATCH_SIZE)

7725


In [ ]:
with strategy.scope() as scope:

    bert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased',output_attentions = False,return_dict=False)
    # we do not need attention outputs
    # we want to return tuples since they are easier to access

    bert_model.trainable = False
    # setting trainable to false ensures
    # we do not update its weights

    # Define the learning rate schedule parameters
    initial_learning_rate = 0.001
    decay_rate = 0.95
    decay_steps = 1000

    # Create an exponential decay learning rate schedule
    lr_schedule = ExponentialDecay(
        initial_learning_rate,
        decay_steps=decay_steps,
        decay_rate=decay_rate,
        staircase=True  # Optional: Whether to apply decay in a staircase manner
    )

    optimizer = Adam(learning_rate=lr_schedule)

    model_ = tf.keras.Sequential([
        bert_model,
        tf.keras.layers.Lambda(lambda x: x[0][:,0,:]), # https://keras.io/api/layers/core_layers/lambda/
        tf.keras.layers.Dense(64,activation="relu"),
        tf.keras.layers.Dense(32,activation="relu"),
        tf.keras.layers.Dense(10,activation="relu"),
        tf.keras.layers.Dense(3,activation="softmax") # we have 3 classes
    ])

    # Define a checkpoint callback to save the best model
    checkpoint = ModelCheckpoint('distilbert_trained_model.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)

    model_.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', precision, recall, f1])

    # Use the first batch of training data we got to instantiate -- needed for loading weights
    model_(tmp_batch_x) 
    
    #Load model if needed
    #model_.load_weights('transformer_weights.h5')
    # Define a callback to collect metrics history
    history = History()

    model_.fit(
        train_data,
        epochs=2,
        batch_size=BATCH_SIZE,
        steps_per_epoch=steps_per_epoch,
        validation_data=val_data,
        validation_steps=validation_steps,
        validation_batch_size=BATCH_SIZE,
        callbacks=[history, checkpoint]
    )


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Epoch 1/2
 174/7725 [..............................] - ETA: 2:13:06 - loss: 0.5664 - accuracy: 0.7651 - precision: 0.7971 - recall: 0.6592 - f1: 0.6972

In [ ]:
#save our weights -- CANNOT SAVE MODEL BECAUSE OF LAMBDA LAYER
model_.save_weights('transformer_weights.h5')

In [ ]:
# Access the metrics history
print(history.history.keys())  # To see what metrics are available

In [ ]:
import numpy as np

# Access the metrics history
epochs = list(range(1, len(history.history['accuracy']) + 1))  # Assuming all metrics have the same length
precision = np.array(history.history['precision'])
recall = np.array(history.history['recall'])
f1 = np.array(history.history['f1'])
accuracy = np.array(history.history['accuracy'])
loss = np.array(history.history['loss'])
val_accuracy = np.array(history.history['val_accuracy'])
val_loss = np.array(history.history['val_loss'])


In [ ]:
# Plot all four metrics on one graph
plt.figure(figsize=(10, 5))
plt.plot(epochs, val_loss, label='Validation Loss')
plt.plot(epochs, val_accuracy, label='Validation Accuracy')
plt.plot(epochs, loss, label='Training Loss')
plt.plot(epochs, accuracy, label='Training Accuracy')

plt.title('Metrics across Epochs')
plt.xlabel('Epoch')
plt.xticks(epochs)
plt.legend()
plt.savefig("Transformer_BERT_training_val_loss_acc(1).pdf")  # Save the plot before showing
plt.show()


In [ ]:
# Plot all four metrics on one graph
plt.figure(figsize=(10, 5))
plt.plot(epochs, precision, label='Precision')
plt.plot(epochs, recall, label='Recall')
plt.plot(epochs, f1, label='F1 Score')
plt.plot(epochs, accuracy, label='Accuracy')

plt.title('Metrics across Epochs')
plt.xlabel('Epoch')
plt.xticks(epochs)
plt.legend()
plt.savefig("Transformer_BERT_training_PRFA(1).pdf")
plt.show()